In [ ]:
#%matplotlib inline
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Predicción de fallas 

En este cuadernillo utilizaremos un dataset sintético que simula un equipo industrial. Más precisamente el dataset tiene lecturas de sensor de esta máquina simulada. El dataset fue publicado en la [Conferencia de IA para industrias 2020](https://ieeexplore.ieee.org/document/9253083) y fue obtenido del [repositorio UCI](https://archive.ics.uci.edu/ml/datasets/AI4I+2020+Predictive+Maintenance+Dataset)

Primero exploraremos el dataset, luego entrenaremos un modelo predictivo y finalmente analizaremos los resultados obtenidos.

## Exploración de los datos

El dataset se distribuye en formato `csv`. Podemos trabajar  con este tipo de tablas [en Python utilizando la librería pandas](https://phuijse.github.io/PythonBook/contents/pandas/part2.html) como se muestra a continuación

In [ ]:
df = pd.read_csv("data/predictive_maintenance.csv", index_col=0).drop(["Product ID", "Target"], axis=1)
df.head(5)

Pandas retorna un objeto de tipo `DataFrame` el cual tiene métodos y atributos muy útiles para hacer exploración. Por ejemplo:

In [ ]:
df.describe()

De la tabla notamos que hay dos columnas categóricas, una es el tipo de la máquina:

In [ ]:
df['Type'].value_counts()

Y la segunda es el tipo de falla:

In [ ]:
df['Failure Type'].value_counts() 

Existe un importante desbalance en los tipos de falla. Como es de esperarse, la información más abundante corresponde a la máquina en operación normal (`No Failure`)


A continuación si visualiza el coeficiente de correlación entre los atributos numéricos

In [ ]:
df.corr()

- La temperatura ambiente y de la máquina tienen alta correlación
- El torque y la velocidad rotacional están anti correlacionados


A continuación se realiza [una gráfica de dispersión con matplotlib](https://phuijse.github.io/PythonBook/contents/visualization/matplotlib1.html) de las columnas Torque y velocidad rotacional, donde se colorea según el tipo de falla

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6), tight_layout=True)

for failure in df['Failure Type'].unique():
    mask = df['Failure Type'] == failure
    ax.scatter(df["Torque [Nm]"].loc[mask], 
               df["Rotational speed [rpm]"].loc[mask],
               label=failure, s=10, alpha=0.25)
ax.legend()
ax.set_xlabel('Torque')
ax.set_ylabel('Velocidad rotacional');

> Esta gráfica nos indica que estos dos atributos son bastante relevantes para clasificar entre falla y no falla

A continuación preparemos los datos para entrenar un clasificador

## Preparación de los datos

Antes de entrenar un modelo de clasificación necesitamos [preprocesar los datos](https://phuijse.github.io/MachineLearningBook/contents/supervised_learning/features.html)

En este caso normalizaremos los datos numéricos y convertiremos a one-hot los datos categóricos:

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

features = StandardScaler().fit_transform(df.select_dtypes(include=[np.number]).values)
type_one_hot = OneHotEncoder(sparse=False).fit_transform(df["Type"].values.reshape(-1,1))
features = np.concatenate((features, type_one_hot), axis=1)
features.shape

Luego convertimos la etiqueta en un valor numérico (categórico)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
label = le.fit_transform(df['Failure Type'])
np.unique(label, return_counts=True)

Para simplificar el problema agruparemos las clases de falla en tan sólo una clase:

In [ ]:
label_binary = np.zeros_like(label)
label_binary[label==1] = 1
np.unique(label_binary, return_counts=True)

Separamos una partición de test para evaluar el desempeño de nuestro clasificador. Es importante hacerlo de forma estratificada por clase para asegurar de tener ejemplos de la clase minoritaria en test

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, label_binary, 
                                                    stratify=label_binary, test_size=0.3, random_state=12345)
X_train.shape, X_test.shape

## Entrenamiento y validación de modelo

Existe muchos modelos de machine learning para hacer clasificación, como por ejemplo el [regresor logístico](https://phuijse.github.io/MachineLearningBook/contents/supervised_learning/logistic.html), la [máquina de soporte vectorial](https://phuijse.github.io/MachineLearningBook/contents/supervised_learning/svm.html) y los [árboles de decisión](https://phuijse.github.io/MachineLearningBook/contents/supervised_learning/trees.html). Lo más correcto sería probar los distintos modelos y mantener el que obtenga mejor desempeño en test.

Sin embargo, para simplificar, nos concentraremos en modelos de tipo [ensamble paralelo de árboles de decisión](https://phuijse.github.io/MachineLearningBook/contents/supervised_learning/ensembles1.html) que suelen tener buen desempeño con datos tabulares. 

Se realiza una [validación cruzada](https://phuijse.github.io/MachineLearningBook/contents/supervised_learning/validation.html) estratificada para escoger los hiperparámetros del modelo, a modo de ejemplo se exploran distintos valores para 

- la cantidad de árboles: `n_estimators`
- el criterio utilizado para separar los nodos del los árboles: `criterion`

Para combatir el desbalance de clases tomamos dos precauciones

- Se utiliza ponderación para "aumentar la importancia" de la clase minoritaria
- Se utiliza una métrica robusta al desbalance para hacer la validación cruzada: área bajo la curva ROC

Otras opción más avanzada sería hacer [aumentación sintética de la clase minoritaria](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

model = RandomForestClassifier(class_weight='balanced', max_depth=None, random_state=12345)
params = {'n_estimators': [10, 20, 50, 100], 'criterion': ['gini', 'entropy']}
validator = GridSearchCV(model, params, cv=5, n_jobs=4, scoring='f1')
validator.fit(X_train, y_train)

pd.DataFrame(validator.cv_results_).sort_values(by="rank_test_score").head(5)

## Evaluación del modelo

Evaluemos el desempeño del clasificador en el conjunto de test

En problemas de dos clases podemos evaluar graficamente utilizando curvas de [curva de desempeño](https://phuijse.github.io/MachineLearningBook/contents/supervised_learning/metrics.html)

In [ ]:
from sklearn.metrics import precision_recall_curve, f1_score

best_rf = validator.best_estimator_
y_probs = best_rf.predict_proba(X_test)[:, 1]

fig, ax = plt.subplots(1, 2, figsize=(10, 4), tight_layout=True)
prec, rec, th = precision_recall_curve(y_test, y_probs)
ax[0].plot(rec, prec)
ax[0].set_xlabel('Recall')
ax[0].set_ylabel('Precision')

f1_score = (prec[:-1]*rec[:-1])/(prec[:-1] + rec[:-1])
ax[1].plot(th, f1_score)
ax[1].set_xlabel('Threshold')
ax[1].set_ylabel('F1-score')

best_th = np.argmax(f1_score)
ax[0].scatter(rec[best_th], prec[best_th], c='k')
ax[1].axvline(th[best_th], c='k', ls='--')
ax[1].axhline(f1_score[best_th], c='k', ls='--');

La matriz de confusión asociada al umbral de operación de máximo f1-score es:

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, classification_report

y_pred = y_probs > th[best_th]
print(classification_report(y_test, y_pred))

fig, ax = plt.subplots(figsize=(5, 4), tight_layout=True)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, normalize=None,
                                        ax=ax, cmap='Blues', colorbar=True);


Notar que podemos tomar un umbral distinto. Por ejemplo si queremos aumentar el número de fallas detectadas a costa de tener muchas falsas alarmas:

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, classification_report

y_pred = y_probs > 0.83
print(classification_report(y_test, y_pred))

fig, ax = plt.subplots(figsize=(5, 4), tight_layout=True)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, normalize=None,
                                        ax=ax, cmap='Blues', colorbar=True);


## Análisis de relevencia de los atributos

Una ventaja de los ensambles paralelos es que entregan como subproducto un proxy de la releveancia de las características. 

La siguiente figura muestra los atributos ordenados según su relevancia:

In [ ]:
fnames = np.array(list(df.select_dtypes(include=[np.number]).columns) + ['model1', 'model2', 'model3'])
idx =  np.argsort(best_rf.feature_importances_)

fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
ax.barh(y=fnames[idx], width=best_rf.feature_importances_[idx], label='detectadas')
ax.set_xlabel('Ganancia de información promedio');

## Análisis de errores

Una vez que tenemos un modelo que es capaz de generalizar es conveniente estudiar los errores (falsos positivos y falsos negativos)

En este caso utilizaremos como umbral de operación aquel de máximo f1-score

La siguiente figura muestra los ejemplos de falla detectadas (azul) y las que no fueron detectadas (naranjo). Algunas fallas parecen ser más difíciles de predecir que otras

In [ ]:
y_pred = best_rf.predict_proba(features)[:, 1] > th[best_th]
fallas_no_detectadas = (label_binary == 0) & (y_pred == 1)
fallas_detectadas = (label_binary == 0) & (y_pred == 0)

fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
fallas, cantidad = np.unique(le.inverse_transform(label[fallas_detectadas]), return_counts=True)
ax.barh(y=fallas, width=cantidad, label='detectadas')
fallas, cantidad = np.unique(le.inverse_transform(label[fallas_no_detectadas]), return_counts=True)
ax.barh(y=fallas, width=cantidad, label='no detectadas')
ax.legend();